# Preparação dos dados

#### 1 - Carga dos dados no dataframe

In [32]:
import pandas as pd
import numpy as np

# Realizando parse de data em coluna única no momento da carga dos dados no dataframe
df = pd.read_csv("../../Data/Raw/household_power_consumption.txt", sep=";", parse_dates=[['Date','Time']]) 

df.head()

C:\Users\marce\.virtualenvs\Code-P0AkfBoR\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Date_Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16 17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
1,2006-12-16 17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2,2006-12-16 17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
3,2006-12-16 17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
4,2006-12-16 17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


#### 2 - Verificações

In [33]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 8 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   Date_Time              2075259 non-null  datetime64[ns]
 1   Global_active_power    2075259 non-null  object        
 2   Global_reactive_power  2075259 non-null  object        
 3   Voltage                2075259 non-null  object        
 4   Global_intensity       2075259 non-null  object        
 5   Sub_metering_1         2075259 non-null  object        
 6   Sub_metering_2         2075259 non-null  object        
 7   Sub_metering_3         2049280 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 126.7+ MB


> Podemos verificar que Sub_metering_3 contém menos valores não nulos e tem tipo diferente das outras. Pela descrição do problema todas deveriam ser numéricas

In [34]:
df.describe(include='all')

C:\Users\marce\AppData\Local\Temp/ipykernel_1684/2884002236.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe(include='all')


,Date_Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
count,2075259,2075259,2075259,2075259,2075259,2075259,2075259,2.049280e+06
unique,2075259,6534,896,5168,377,153,145,NaN
top,2006-12-16 17:24:00,?,0.000,?,1.000,0.000,0.000,NaN
freq,1,25979,472786,25979,169406,1840611,1408274,NaN
first,2006-12-16 17:24:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2010-12-11 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.458447e+00
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.437154e+00
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00


> Descrição mostra **NaN** para colunas que deveriam ser numéricas, Ex.: Global_active_power, Voltage, etc., possível existência de dados com tipos diferentes

In [35]:
df.dtypes

Date_Time                datetime64[ns]
Global_active_power              object
Global_reactive_power            object
Voltage                          object
Global_intensity                 object
Sub_metering_1                   object
Sub_metering_2                   object
Sub_metering_3                  float64
dtype: object

> Verificação de tipos mostra object para as mesmas colunas, o que reforça a suposição

In [36]:
# Tentativa de converter uma das colunas em float
pd.to_numeric(df['Voltage'])


ValueError: Unable to parse string "?" at position 6839

> Erro dada a impossibilidade de converter a string '?' em número. Verificando o arquivo conseguimos ver que os valores faltantes estão marcado dessa forma.
![missing values](missing_values.png)

#### 3 - Ajustes

In [107]:
# Substitui as strings '?' por np.NaN e as exclui posteriormente
df.replace('?', np.NAN, inplace=True)
df.dropna(inplace=True)

In [108]:
# Converte todas as colunas em float, menos a primeira que é do tipo datetime
for col in df.columns[1:]:
    df[col] = df[col].astype(float)

In [109]:
df.describe(include='all')

C:\Users\marce\AppData\Local\Temp/ipykernel_1684/2884002236.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe(include='all')


,Date_Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Month,Year,Season,Winter,Spring,Summer,Autunum
count,2049280,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06
unique,2049280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2006-12-16 17:24:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
first,2006-12-16 17:24:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,2010-12-11 23:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,1.091615e+00,1.237145e-01,2.408399e+02,4.627759e+00,1.121923e+00,1.298520e+00,6.458447e+00,6.497968e+00,2.008425e+03,2.499732e+00,2.473644e-01,2.536686e-01,2.508374e-01,2.481296e-01
std,NaN,1.057294e+00,1.127220e-01,3.239987e+00,4.444396e+00,6.153031e+00,5.822026e+00,8.437154e+00,3.446016e+00,1.124388e+00,1.113997e+00,4.314804e-01,4.351103e-01,4.334952e-01,4.319275e-01
min,NaN,7.600000e-02,0.000000e+00,2.232000e+02,2.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.006000e+03,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,NaN,3.080000e-01,4.800000e-02,2.389900e+02,1.400000e+00,0.000000e+00,0.000000e+00,0.000000e+00,4.000000e+00,2.007000e+03,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


> Dada a grande quantidade de dados (> 2Mi) vamos agregá-los. De minutos para diário. Devemos tomar cuidado com algumas medidas que não podem ser somadas, como: Voltagem, que será feita a média. 

In [128]:
df_diario = df.groupby(df['Date_Time'].dt.date).agg(Pot_ativa = ('Global_active_power', 'sum'), Pot_reativa = ('Global_reactive_power', 'sum'), Tensao = ('Voltage', 'median'), Corrente = ('Global_intensity', 'median'), Pot_cozinha = ('Sub_metering_1', 'sum'), Pot_lavanderia = ('Sub_metering_2', 'sum'), Pot_Porao = ('Sub_metering_3', 'sum'))


In [129]:
# Reindexando o dataframe para ter a coluna de data disponível para manipulação
df_diario.reset_index(inplace=True)

In [130]:
# Convertendo a coluna Date_Time em datetime64
df_diario['Date_Time'] = df_diario['Date_Time'].astype(np.datetime64)

df_diario.dtypes

Date_Time         datetime64[ns]
Pot_ativa                float64
Pot_reativa              float64
Tensao                   float64
Corrente                 float64
Pot_cozinha              float64
Pot_lavanderia           float64
Pot_Porao                float64
dtype: object

#### 4 - Enriquecendo os dados com novas categorias (feature engenieering)

> Criando novas colunas de ano e mes. Podem ser importantes para padrões de consumos por mês ou ano.

In [131]:
df_diario['Mes'] = df_diario['Date_Time'].dt.month

df_diario['Ano'] = df_diario['Date_Time'].dt.year

df_diario.head(3)

,Date_Time,Pot_ativa,Pot_reativa,Tensao,Corrente,Pot_cozinha,Pot_lavanderia,Pot_Porao,Mes,Ano
0,2006-12-16,1209.176,34.922,235.915,13.8,0.0,546.0,4926.0,12,2006
1,2006-12-17,3390.460,226.006,240.315,9.8,2033.0,4187.0,13341.0,12,2006
2,2006-12-18,2203.826,161.792,241.470,6.2,1063.0,2621.0,14018.0,12,2006


> Criando coluna para as estações do ano. Partiremos da premissa que as medições são realizadas no hemisfério norte e que só há mudanças de estação no início dos meses. Podem ser importantes para padrões de consumos por estação.
> 
> - Dez -> Fev = Inverno (1)<br>
> - Mar -> Mai = Primavera (2)<br> 
> - Jun -> Ago = Verão (3)<br>
> - Set -> Nov = Outono (4)<br>

In [132]:
seasons = [1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 1]

month_to_season = dict(zip(range(1,13), seasons))

df_diario['Estacao_Ano']= df_diario['Mes'].map(month_to_season) 

df_diario.head(3)

,Date_Time,Pot_ativa,Pot_reativa,Tensao,Corrente,Pot_cozinha,Pot_lavanderia,Pot_Porao,Mes,Ano,Estacao_Ano
0,2006-12-16,1209.176,34.922,235.915,13.8,0.0,546.0,4926.0,12,2006,1
1,2006-12-17,3390.460,226.006,240.315,9.8,2033.0,4187.0,13341.0,12,2006,1
2,2006-12-18,2203.826,161.792,241.470,6.2,1063.0,2621.0,14018.0,12,2006,1


> Como não existe diferença de importância entre as estações do ano, vamos gerar uma coluna para cada uma. Isso evita de que as distâncias calculadas pelos algoritmos de clusterização não deem mais importância a uma do que a outra.
> Neste caso a columa Season foi gerada por um label encode e as novas serão one hot encode.

In [133]:
df_diario['Inverno'] = df_diario['Estacao_Ano'].apply(lambda x: 1 if x == 1 else 0)
df_diario['Primavera'] = df_diario['Estacao_Ano'].apply(lambda x: 1 if x == 2 else 0)
df_diario['Verao'] = df_diario['Estacao_Ano'].apply(lambda x: 1 if x == 3 else 0)
df_diario['Outono'] = df_diario['Estacao_Ano'].apply(lambda x: 1 if x == 4 else 0)

df_diario.head(3)

,Date_Time,Pot_ativa,Pot_reativa,Tensao,Corrente,Pot_cozinha,Pot_lavanderia,Pot_Porao,Mes,Ano,Estacao_Ano,Inverno,Primavera,Verao,Outono
0,2006-12-16,1209.176,34.922,235.915,13.8,0.0,546.0,4926.0,12,2006,1,1,0,0,0
1,2006-12-17,3390.460,226.006,240.315,9.8,2033.0,4187.0,13341.0,12,2006,1,1,0,0,0
2,2006-12-18,2203.826,161.792,241.470,6.2,1063.0,2621.0,14018.0,12,2006,1,1,0,0,0


In [134]:
# Exclusão da coluna Date_Time e Season que não são mais necessárias

df_diario.drop(['Estacao_Ano'], inplace=True, axis=1)
df_diario.drop(['Date_Time'], inplace=True, axis=1)

In [135]:
df_diario.head()

,Pot_ativa,Pot_reativa,Tensao,Corrente,Pot_cozinha,Pot_lavanderia,Pot_Porao,Mes,Ano,Inverno,Primavera,Verao,Outono
0,1209.176,34.922,235.915,13.8,0.0,546.0,4926.0,12,2006,1,0,0,0
1,3390.460,226.006,240.315,9.8,2033.0,4187.0,13341.0,12,2006,1,0,0,0
2,2203.826,161.792,241.470,6.2,1063.0,2621.0,14018.0,12,2006,1,0,0,0
3,1666.194,150.942,242.200,2.2,839.0,7602.0,6197.0,12,2006,1,0,0,0
4,2225.748,160.998,243.030,5.4,0.0,2648.0,14063.0,12,2006,1,0,0,0


> Analisando a parte elétrica. Foram informadas as potências ativas e reativas totais, mas não a aparente total. Pelo triângulo de potências a potência aparente total pode ser descoberta, mas para a nossa análise não fará a diferença. Um consumidor residencial paga pela potência ativa consumida. A potência reativa tem mais a ver com a qualidade do consumo da potência total e só seria importante se o consumidor fosse industrial. No Brasil, por lei, a quantidade de energia reativa retornada à geração não pode ser maior que 8%. Vamos ignorá-la neste caso.

![Triangulo das Potências](triangulo_potencias.png)  fórmula ![Fórmula das Potências](formula_potencias.png)

Analogia

![Analogia](analogia_chopp.png) 

#### 5 - Salvando o dataframe preparado para a análise exploratória

In [136]:
df_diario.to_pickle("../../Data/Processed/df_prepared.pkl")